In [5]:
import sys
sys.path.append('../')

import numpy as np
import time
import copy
from tqdm import tqdm
import os


import importlib

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import Normalize
import numpy as np
import numpy.ma as ma

from collections import OrderedDict

import torch
from torch.optim import SGD, Adam, lr_scheduler
device = 'cuda'

import importlib

import TheoryCurves
importlib.reload(TheoryCurves)

import auxFuncs
importlib.reload(auxFuncs)

import pytz
from datetime import datetime
from dateutil.relativedelta import relativedelta
tz = pytz.timezone('US/Eastern')

def time_now():
    return datetime.now(tz)

def time_diff(t_a, t_b):
    t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

# Choose Parameters:

In [6]:
spacing = .01
lam = 0
alphaLog = False

In [7]:
colors = ['tab:blue', 'tab:red', 'tab:green', 'tab:purple', 'tab:orange', 'tab:brown', 'tab:pink', 'tab:olive', 'tab:cyan']
linestyles = ['-', '--', ':', '-.']
markerstyles = ['.','*','x']

In [8]:
#Width and height of axes.
w=2.5
h=2
plt.rcParams.update({'font.size': 12, 'figure.figsize': [w,h]})

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams['legend.title_fontsize'] = 'small'

#Helper Function for setting figure size by axis size.
#set_size(w,h,ax)
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

# Homogeneous Readouts Changing Number of Readouts
## Collapsed Phase Diagram
What's important are the effective noise to signal ratios $H = \frac{\eta^2}{s(1-c)}$, $W = \frac{\omega^2}{s(1-c)}$, $Z = \frac{\zeta^2}{s(1-c)}$. Also the sample complexity $\alpha$, and the "code-task alignment" which is in this case parameterized by $\rho$.

In [9]:
#Plotting against sample complexity.
if alphaLog:
    alphas = np.append(np.logspace(-2.1, 1, 301), np.inf)
else:
    alphas = np.append(np.linspace(0, 3, 301), np.inf)

ks = np.array(list(range(1, 101)) + [np.inf])
numKs = len(ks)

rhos = [0, .3, .6]
numRhos = len(rhos)

lam = 0
#Global not working yet -- instabilities in the calculation of optimal regularization.

H_spacing = spacing
H_max = 2.1
H_list = np.linspace(0, H_max, int(H_max/H_spacing)+1)

W_spacing = spacing
W_max = 2.1
W_list = np.linspace(0, W_max, int(W_max/W_spacing)+1)

Z_spacing = spacing
Z_max = 2.1
Z_list = np.linspace(0, Z_max, int(Z_max/Z_spacing)+1)

#These parameters will not affect the phase diagrams
s = 1
c = .1 #As long as we have c>0 this should give the right answers.

errors_H = np.empty((len(rhos), len(H_list), len(alphas), numKs))
errors_W = np.empty((len(rhos), len(W_list), len(alphas), numKs))
errors_Z = np.empty((len(rhos), len(Z_list), len(alphas), numKs))

nu_0 = 1

In [13]:
for HInd, H in enumerate(H_list):
    for rhoInd, rho in enumerate(rhos):
        for kInd, k in enumerate(ks):
            eta = (H*s*(1-c))**(1/2)
            omega = 0
            zeta = 0
            errors_H[rhoInd, HInd, :, kInd] = TheoryCurves.getEquiCorrErrorCurve_Homog_Exclusive(alphas, k, c, nu_0, lam, zeta, eta, s = s, omega = omega, rho = rho)
        
np.save(f'errors_H_lam{lam}.npy', errors_H)

In [14]:
for WInd, W in enumerate(W_list):
    for rhoInd, rho in enumerate(rhos):
        for kInd, k in enumerate(ks):
            eta = 0
            omega = (W*s*(1-c))**(1/2)
            zeta = 0
            errors_W[rhoInd, WInd, :, kInd] = TheoryCurves.getEquiCorrErrorCurve_Homog_Exclusive(alphas, k, c, nu_0, lam, zeta, eta, s = s, omega = omega, rho = rho)
            
np.save(f'errors_W_lam{lam}.npy', errors_W)

In [15]:
for ZInd, Z in enumerate(Z_list):
    for rhoInd, rho in enumerate(rhos):
        for kInd, k in enumerate(ks):
            eta = 0
            omega = 0
            zeta = (Z*s*(1-c))**(1/2)
            errors_Z[rhoInd, ZInd, :, kInd] = TheoryCurves.getEquiCorrErrorCurve_Homog_Exclusive(alphas, k, c, nu_0, lam, zeta, eta, s = s, omega = omega, rho = rho)
np.save(f'errors_Z_lam{lam}.npy', errors_Z)